In [2]:
import os
import zipfile
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import tensorflow_addons as tfa
from PIL import Image, ImageFilter
from tqdm import tqdm
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path

2022-02-13 11:30:59.350237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/shinya/DataScience/data/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-13 11:30:59.350299: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [464]:
def unzip_dataset(INPATH, OUTPATH):
    with zipfile.ZipFile(INPATH) as zf:
        zf.extractall(OUTPATH)
    
unzip_dataset(INPATH = './train_data.zip', OUTPATH = './')
unzip_dataset(INPATH = './test_data.zip', OUTPATH = './')

In [465]:
train_dir = '../Casting_products_pytorch/train_data/'
test_dir = '../Casting_products_pytorch/test_data/'
train_df = pd.read_csv('../Casting_products_pytorch/train.csv')

In [466]:
train_df, val_df = train_test_split(np.array(train_df), test_size=0.1)

In [467]:
train_df

array([['cast_def_0_61.jpeg', 0],
       ['cast_def_0_2411.jpeg', 0],
       ['cast_def_0_19.jpeg', 0],
       ['cast_def_0_24.jpeg', 0],
       ['cast_def_0_97.jpeg', 0],
       ['cast_def_0_9.jpeg', 0],
       ['cast_def_0_105.jpeg', 0],
       ['cast_ok_0_248.jpeg', 1],
       ['cast_ok_0_379.jpeg', 1],
       ['cast_def_0_163.jpeg', 0],
       ['cast_def_0_92.jpeg', 0],
       ['cast_def_0_2470.jpeg', 0],
       ['cast_ok_0_355.jpeg', 1],
       ['cast_ok_0_402.jpeg', 1],
       ['cast_def_0_2489.jpeg', 0],
       ['cast_def_0_192.jpeg', 0],
       ['cast_ok_0_175.jpeg', 1],
       ['cast_ok_0_434.jpeg', 1],
       ['cast_def_0_110.jpeg', 0],
       ['cast_def_0_56.jpeg', 0],
       ['cast_def_0_333.jpeg', 0],
       ['cast_def_0_135.jpeg', 0],
       ['cast_def_0_136.jpeg', 0],
       ['cast_def_0_359.jpeg', 0],
       ['cast_ok_0_364.jpeg', 1],
       ['cast_def_0_320.jpeg', 0],
       ['cast_def_0_74.jpeg', 0],
       ['cast_ok_0_171.jpeg', 1],
       ['cast_def_0_89.jpeg', 0],


In [468]:
val_df

array([['cast_def_0_2400.jpeg', 0],
       ['cast_def_0_2426.jpeg', 0],
       ['cast_def_0_203.jpeg', 0],
       ['cast_ok_0_250.jpeg', 1],
       ['cast_def_0_2405.jpeg', 0],
       ['cast_def_0_109.jpeg', 0],
       ['cast_def_0_2482.jpeg', 0],
       ['cast_def_0_11.jpeg', 0],
       ['cast_ok_0_302.jpeg', 1],
       ['cast_def_0_2467.jpeg', 0],
       ['cast_def_0_330.jpeg', 0],
       ['cast_def_0_2471.jpeg', 0],
       ['cast_def_0_99.jpeg', 0],
       ['cast_ok_0_220.jpeg', 1],
       ['cast_ok_0_435.jpeg', 1],
       ['cast_def_0_133.jpeg', 0],
       ['cast_def_0_2527.jpeg', 0],
       ['cast_def_0_2490.jpeg', 0],
       ['cast_def_0_360.jpeg', 0],
       ['cast_def_0_148.jpeg', 0],
       ['cast_ok_0_154.jpeg', 1],
       ['cast_def_0_41.jpeg', 0],
       ['cast_def_0_2499.jpeg', 0],
       ['cast_def_0_116.jpeg', 0],
       ['cast_def_0_2403.jpeg', 0]], dtype=object)

In [469]:
transform = torchvision.transforms.ToTensor()

In [470]:
class Casting_products_Datasets(Dataset):
    def __init__(self, dir_name, csv_file, transform):
        self.dir_name = dir_name
        self.csv_file = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, index):
        file = self.csv_file[0][index]
        label = torch.LongTensor(np.array([self.csv_file[index][1]]))
        image = Image.open(self.dir_name + file)
        image = self.transform(image)

        return image, label

In [471]:
train_dataset = Casting_products_Datasets(train_dir, train_df, transform)
val_dataset = Casting_products_Datasets(train_dir, val_df, transform)

In [472]:
a, b = train_dataset[0]
print(a.size(), b.size())
print(a, b)

torch.Size([3, 300, 300]) torch.Size([1])
tensor([[[0.6706, 0.6706, 0.6706,  ..., 0.5804, 0.5765, 0.5725],
         [0.6667, 0.6667, 0.6706,  ..., 0.5804, 0.5765, 0.5725],
         [0.6627, 0.6627, 0.6667,  ..., 0.5804, 0.5725, 0.5686],
         ...,
         [0.7098, 0.7098, 0.7059,  ..., 0.6039, 0.6039, 0.6039],
         [0.7098, 0.7098, 0.7059,  ..., 0.6000, 0.6000, 0.6000],
         [0.7098, 0.7098, 0.7059,  ..., 0.5922, 0.5922, 0.5922]],

        [[0.6706, 0.6706, 0.6706,  ..., 0.5804, 0.5765, 0.5725],
         [0.6667, 0.6667, 0.6706,  ..., 0.5804, 0.5765, 0.5725],
         [0.6627, 0.6627, 0.6667,  ..., 0.5804, 0.5725, 0.5686],
         ...,
         [0.7098, 0.7098, 0.7059,  ..., 0.6039, 0.6039, 0.6039],
         [0.7098, 0.7098, 0.7059,  ..., 0.6000, 0.6000, 0.6000],
         [0.7098, 0.7098, 0.7059,  ..., 0.5922, 0.5922, 0.5922]],

        [[0.6706, 0.6706, 0.6706,  ..., 0.5804, 0.5765, 0.5725],
         [0.6667, 0.6667, 0.6706,  ..., 0.5804, 0.5765, 0.5725],
         [0.6627

In [473]:
batch_size = 32

In [474]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)